In [1]:
from theFuncs import *

# DATA

In [2]:
# load_DTB3_SPY().to_csv('./data/spy_dtb3.csv')

In [3]:
sp100 = os.listdir(os.getcwd()+'/data/stock_dfs')
sp100_tickers = []
for csvfile in sp100:
    sp100_tickers.append(csvfile[:-4])

# sp100_tickers

In [4]:
ticker_data = get_ticker_data_multisource(sp100_tickers)

In [5]:
# ticker_data.loc['CSCO']

In [6]:
model = RollingOLS.from_formula(
        "(ret - DTB3) ~ (spy_ret - DTB3)", 
        data = ticker_data.loc[sp100_tickers[0]], 
        window = 66
    ).fit(params_only=True)
df = model.params
df.columns = ["alpha", "Beta"]
df['ticker'] = sp100_tickers[0]
df.set_index('ticker', inplace=True, append=True)
df = df.reorder_levels(['ticker', 'date']).sort_index()


for ticker in sp100_tickers[1:]:
    model = RollingOLS.from_formula(
        "(ret - DTB3) ~ (spy_ret - DTB3)", 
        data = ticker_data.loc[ticker], 
        window = 66
    ).fit(params_only=True)

    temp = model.params
    temp.columns = ["alpha", "Beta"]
    temp['ticker'] = ticker
    temp.set_index('ticker', inplace=True, append=True)
    temp = temp.reorder_levels(['ticker', 'date']).sort_index()
    df = pd.concat(
        [ 
            df, 
            temp
        ]
    )

ticker_data = ticker_data.join(df)

ticker_data.head()

high        low       open      close       volume  \
ticker date                                                                  
AAPL   2019-01-03  36.430000  35.500000  35.994999  35.547501  365248800.0   
       2019-01-04  37.137501  35.950001  36.132500  37.064999  234428400.0   
       2019-01-07  37.207500  36.474998  37.174999  36.982498  219111200.0   
       2019-01-08  37.955002  37.130001  37.389999  37.687500  164101200.0   
       2019-01-09  38.632500  37.407501  37.822498  38.327499  180396400.0   

                        ret   spy_close    DTB3   spy_ret  alpha  Beta  
ticker date                                                             
AAPL   2019-01-03 -0.099607  232.998627  0.0236 -0.023863    NaN   NaN  
       2019-01-04  0.042689  240.803085  0.0237  0.033496    NaN   NaN  
       2019-01-07 -0.002226  242.701736  0.0241  0.007885    NaN   NaN  
       2019-01-08  0.019063  244.981995  0.0241  0.009395    NaN   NaN  
       2019-01-09  0.016982  246.126907  0.0240  0.004673    NaN   NaN

In [7]:
market_caps = pd.DataFrame(
    ticker_data.reset_index(level=[0, 1]).groupby(['ticker', 'date']).apply(
        lambda x: x.close * x.volume
    )
)
market_caps.reset_index(level=2, drop=True, inplace=True)
market_caps.rename(columns={0: "MarketCap"}, inplace=True)

In [8]:
market_caps.head()

MarketCap
ticker date                    
AAPL   2019-01-03  1.298368e+10
       2019-01-04  8.689088e+09
       2019-01-07  8.103280e+09
       2019-01-08  6.184564e+09
       2019-01-09  6.914143e+09

In [9]:
market_caps.head()

MarketCap
ticker date                    
AAPL   2019-01-03  1.298368e+10
       2019-01-04  8.689088e+09
       2019-01-07  8.103280e+09
       2019-01-08  6.184564e+09
       2019-01-09  6.914143e+09

In [10]:
ticker_data_wide = ticker_data.reset_index(level=['ticker', 'date'])
ticker_data_wide = ticker_data_wide[['ticker', 'date', 'ret']].pivot(
    index='date', 
    columns='ticker', 
    values='ret'
)
ticker_data_wide.head()

ticker,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMGN,AMT,AMZN,AVGO,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.099607,-0.032949,-0.047194,-0.034142,-0.039498,-0.021959,-0.015216,0.005997,-0.025242,-0.088951,...,-0.027270,-0.037016,-0.028392,-0.014024,-0.036037,0.003570,-0.010435,-0.007882,-0.005142,-0.015354
2019-01-04,0.042689,0.032217,0.028541,0.038884,0.048632,0.036903,0.034184,0.003995,0.050064,0.009828,...,0.011695,0.038514,0.034833,0.024726,0.043081,0.002490,0.033269,0.029633,0.006246,0.036870
2019-01-07,-0.002226,0.014595,0.014976,0.003473,0.013573,0.005724,0.013457,0.006190,0.034353,0.020366,...,0.001920,0.006241,-0.003376,-0.004698,0.018032,0.006388,0.005750,-0.006465,0.011772,0.005200
2019-01-08,0.019063,0.004648,-0.012006,0.025290,0.014918,0.003959,0.012824,0.011300,0.016612,-0.008026,...,0.013371,0.087270,0.000103,0.006436,0.005439,0.029267,0.020009,-0.002099,0.006981,0.007271
2019-01-09,0.016982,-0.032823,0.016691,0.005030,0.011819,0.014543,-0.001196,-0.008753,0.001714,0.043250,...,0.001439,-0.002587,0.005132,0.010659,0.011769,-0.022782,0.012050,0.005469,-0.003256,0.005275


In [11]:
rolling_correlations = ticker_data_wide.rolling(66).corr().dropna()
rolling_correlations.head()

ticker                 AAPL      ABBV       ABT       ACN      ADBE       AIG  \
date       ticker                                                               
2019-06-24 AAPL    1.000000  0.296962  0.224228  0.354141  0.403017 -0.057218   
           ABBV    0.296962  1.000000  0.495805  0.338759  0.345130 -0.007118   
           ABT     0.224228  0.495805  1.000000  0.447644  0.510348  0.037750   
           ACN     0.354141  0.338759  0.447644  1.000000  0.527017  0.108172   
           ADBE    0.403017  0.345130  0.510348  0.527017  1.000000  0.123667   

ticker                 AMGN       AMT      AMZN      AVGO  ...       UNH  \
date       ticker                                          ...             
2019-06-24 AAPL    0.248007 -0.039255  0.561013  0.606993  ...  0.117699   
           ABBV    0.515949  0.102796  0.376902  0.127400  ...  0.410813   
           ABT     0.539751  0.297049  0.391182  0.299744  ...  0.388814   
           ACN     0.283967  0.152951  0.498013  0.369200  ...  0.136531   
           ADBE    0.302570  0.199119  0.739261  0.418718  ...  0.211514   

ticker                  UNP       UPS       USB         V        VZ       WBA  \
date       ticker                                                               
2019-06-24 AAPL    0.370165  0.409476  0.366545  0.501574 -0.112949  0.228087   
           ABBV    0.188250  0.153467  0.285019  0.448157  0.079838 -0.087340   
           ABT     0.297078  0.199606  0.363545  0.472988 -0.019647  0.103246   
           ACN     0.385596  0.390947  0.351905  0.621636 -0.238056  0.252542   
           ADBE    0.455503  0.293439  0.379133  0.820151 -0.091270  0.225878   

ticker                  WFC       WMT       XOM  
date       ticker                                
2019-06-24 AAPL    0.301752  0.151945  0.301929  
           ABBV    0.270842  0.111515  0.323539  
           ABT     0.339443  0.308035  0.302968  
           ACN     0.317963  0.330239  0.318322  
           ADBE    0.256738  0.357301  0.303250  

[5 rows x 101 columns]

In [12]:
rolling_covariances = ticker_data_wide.rolling(66).cov().dropna()
rolling_covariances.head()

ticker                 AAPL      ABBV       ABT       ACN      ADBE       AIG  \
date       ticker                                                               
2019-06-24 AAPL    0.000275  0.000051  0.000043  0.000060  0.000111 -0.000013   
           ABBV    0.000051  0.000107  0.000060  0.000036  0.000059 -0.000001   
           ABT     0.000043  0.000060  0.000137  0.000054  0.000099  0.000006   
           ACN     0.000060  0.000036  0.000054  0.000105  0.000089  0.000015   
           ADBE    0.000111  0.000059  0.000099  0.000089  0.000274  0.000028   

ticker                 AMGN       AMT      AMZN      AVGO  ...       UNH  \
date       ticker                                          ...             
2019-06-24 AAPL    0.000049 -0.000006  0.000133  0.000189  ...  0.000032   
           ABBV    0.000063  0.000010  0.000056  0.000025  ...  0.000069   
           ABT     0.000075  0.000034  0.000066  0.000066  ...  0.000074   
           ACN     0.000034  0.000015  0.000073  0.000071  ...  0.000023   
           ADBE    0.000059  0.000033  0.000175  0.000130  ...  0.000057   

ticker                  UNP       UPS       USB         V        VZ       WBA  \
date       ticker                                                               
2019-06-24 AAPL    0.000070  0.000110  0.000058  0.000078 -0.000023  0.000073   
           ABBV    0.000022  0.000026  0.000028  0.000044  0.000010 -0.000017   
           ABT     0.000040  0.000038  0.000041  0.000052 -0.000003  0.000023   
           ACN     0.000045  0.000065  0.000035  0.000060 -0.000030  0.000050   
           ADBE    0.000086  0.000079  0.000060  0.000127 -0.000019  0.000072   

ticker                  WFC       WMT       XOM  
date       ticker                                
2019-06-24 AAPL    0.000061  0.000021  0.000054  
           ABBV    0.000034  0.000009  0.000036  
           ABT     0.000048  0.000030  0.000038  
           ACN     0.000039  0.000028  0.000035  
           ADBE    0.000051  0.000049  0.000054  

[5 rows x 101 columns]

In [13]:
rolling_covariances.loc[('2019-06-24', 'AAPL'), 'AAPL']

date        ticker
2019-06-24  AAPL      0.000275
Name: AAPL, dtype: float64

# CLUSTERING

In [14]:
date = '2019-06-24'
K = 20

In [15]:
x, y, z = clustering_model(rolling_correlations, date, K)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-26
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10303 rows, 10302 columns and 30704 nonzeros
Model fingerprint: 0x351b68d8
Variable types: 0 continuous, 10302 integer (10302 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 33.3704622
Presolve time: 0.05s
Presolved: 10303 rows, 10302 columns, 30704 nonzeros
Variable types: 0 continuous, 10302 integer (10302 binary)
Found heuristic solution: objective 66.1040777

Root relaxation: objective 7.261684e+01, 559 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

# INDEX BY CLUSTER WEIGHTS

In [16]:
z_market_cap = market_caps.loc[
    market_caps.index.get_level_values(1) == datetime.strptime(date, "%Y-%m-%d")
].join(z)
z_market_cap.reset_index(drop=True, level=1, inplace=True)

In [17]:
center_weights = z_market_cap[(z_market_cap.in_center == 1) & (z_market_cap.is_center == 1)].groupby("center").MarketCap.sum() / z_market_cap[(z_market_cap.in_center == 1) & (z_market_cap.is_center == 1)].MarketCap.sum()

In [18]:
center_weights

center
BIIB     0.004954
C        0.129697
CL       0.020114
COST     0.052208
DHR      0.020064
DIS      0.021384
EMR      0.088418
EXC      0.019119
GILD     0.058265
GOOGL    0.042579
HON      0.043295
IBM      0.029051
MSFT     0.231541
NVDA     0.111564
PEP      0.019457
PFE      0.028729
T        0.022649
UNH      0.011556
UPS      0.020675
XOM      0.024681
Name: MarketCap, dtype: float64

In [19]:
portfolio_returns = get_portfolio_returns(
    center_weights, date, ticker_data_wide
)
spy_returns = get_spy_returns(date)

portfolio_returns, spy_returns

(1.0122919577413123, 1.0110079840222566)

In [20]:
portfolio_returns - spy_returns

0.0012839737190557532

# INDEX BY MIN VAR

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=80cc924b-f00e-4408-bf29-ac16ba2efaff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>